# LIN 313 Final Project
**_by Jed Wang_**

In [ ]:
# if True, complete tokenizer; if False, verified tokenizer
complete = False

## Step 4a: Import tokenizer & data
From the previous step, import the JSONs, and import the pretraining data from GitHub.

In [ ]:
!pip install transformers datasets tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login()

In [ ]:
# !rm tokenizer_full.json
!wget -O tokenizer_full.json https://gist.github.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_full.json

--2023-11-28 06:03:32--  https://gist.github.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_full.json
Resolving gist.github.com (gist.github.com)... 20.27.177.113
Connecting to gist.github.com (gist.github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_full.json [following]
--2023-11-28 06:03:32--  https://gist.githubusercontent.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_full.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6471221

In [ ]:
# !rm tokenizer_veri.json
!wget -O tokenizer_veri.json https://gist.github.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_veri.json

--2023-11-28 06:03:34--  https://gist.github.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_veri.json
Resolving gist.github.com (gist.github.com)... 20.27.177.113
Connecting to gist.github.com (gist.github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_veri.json [following]
--2023-11-28 06:03:34--  https://gist.githubusercontent.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_veri.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6196696

In [ ]:
# set to False if you want to reload the data
lazy = True

if not lazy:
  # !rm ./sample_data/corpus.zip
  !wget -O ./sample_data/corpus.zip https://github.com/LeftistTachyon/ChineseDecomp/releases/download/v0.1/corpus.zip
  !(cd sample_data && unzip -o corpus.zip)

Import dataset

In [ ]:
from datasets import load_dataset

if not lazy:
  pretrain_dataset = load_dataset(
      "json",
      data_files={ "train": "sample_data/baiducorpus_conv.jsonl" },
      num_proc=4
  )
  pretrain_dataset["train"].num_rows, pretrain_dataset["train"]._indices

## Step 5: Perform the training
This is done with HuggingFace's trainer, with [Peter, Moshe, & Omer](https://arxiv.org/abs/2104.07705)'s hyperparameters.

In [ ]:
!pip install -U transformers
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.1 MB/s eta 0:00:00


Most of these training args are pulled from either [this academic paper](https://arxiv.org/abs/2104.07705) or [HuggingFace's guide on training on a single GPU](https://huggingface.co/docs/transformers/v4.35.2/en/perf_train_gpu_one).

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="pretrain",
    push_to_hub=True,

    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_epsilon=1e-6,
    weight_decay=0.01,

    gradient_accumulation_steps=16,
    per_device_train_batch_size=16,
    num_train_epochs=1.0,

    learning_rate=2e-3,
    warmup_ratio=0.02,
    # max_steps=562,
    fp16=True, # enable on GPUs
    # torch_compile=True,
    # dataloader_num_workers=2,

    disable_tqdm=False
)

In [ ]:
from transformers import PreTrainedTokenizerFast

# Instantiate the tokenizer from before
tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="tokenizer_full.json" if complete else "tokenizer_veri.json",
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
    model_max_length=512
)

In [ ]:
from transformers import AutoModelForMaskedLM

# I was having trouble getting base BERT to work, so here's DistilRoBERTa
model = AutoModelForMaskedLM.from_pretrained("distilroberta-base")

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Step 5.1: Prepare the dataset
Most of the steps taken here are either from [the preparing dataset guide](https://huggingface.co/docs/transformers/training#prepare-a-dataset) or [the MLM guide](https://huggingface.co/docs/transformers/v4.35.2/en/tasks/masked_language_modeling).

In [ ]:
if not lazy:
  shuffled = pretrain_dataset.shuffle(seed=42)
  # filtered = shuffled.filter(lambda _ex, idx: 3 <= idx % 45 <= 4, with_indices=True)
  # filtered = shuffled.filter(lambda _ex, idx: idx % 100 == 42, with_indices=True)
  filtered = shuffled.filter(lambda _exs, idxs: [idx % 85 < 24 for idx in idxs], batched=True, with_indices=True, num_proc=8)

In [ ]:
import multiprocessing

multiprocessing.cpu_count()

8

In [ ]:
def tokenize_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = load_dataset("jedwang/baidu-baike-tokenized",
                                 revision="quart",
                                 num_proc=multiprocessing.cpu_count()) \
    if lazy \
    else filtered.map(tokenize_function,
                      batched=True,
                      num_proc=multiprocessing.cpu_count())

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1016664 [00:00<?, ? examples/s]

In [ ]:
if not lazy:
  tokenized_dataset.push_to_hub(
      repo_id="jedwang/baidu-baike-tokenized",
      revision="quart"
  )

Instantiate the trainer (<https://huggingface.co/docs/transformers/v4.35.2/en/main_classes/trainer>)

In [ ]:
from transformers import DataCollatorForLanguageModeling

# tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm_probability=0.15
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    # eval_dataset=pretrain_dataset["eval"], # we're doing 0 eval
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Sanity check:

In [ ]:
enc = tokenizer.encode("我们家乡安徽歙县风景秀丽")
# enc.insert(2, 5)
enc, [tokenizer.decode(tok) for tok in enc], tokenizer.decode(enc)

([2,
  5402,
  10993,
  1995,
  4129,
  6505,
  3201,
  3384,
  3746,
  7911,
  1839,
  3196,
  3993,
  7262,
  5202,
  4235,
  3],
 ['[CLS]',
  '我们',
  '家',
  '乡',
  '安',
  '微',
  '幺小',
  '合',
  '羽',
  '欠',
  '且',
  '厶',
  '风',
  '景',
  '秀',
  '丽',
  '[SEP]'],
 '[CLS]我们家乡安徽歙县风景秀丽[SEP]')

Shoutouts to [#107960](https://github.com/pytorch/pytorch/issues/107960) for the following fix:

In [ ]:
# https://github.com/pytorch/pytorch/issues/107960
# !ldconfig -p | grep libcuda
# !find /usr -name 'libcuda.so'
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

Here's the fun part:

In [ ]:
trainer.train()

Step,Training Loss
500,3.767200
1000,2.776700
1500,2.440500
2000,2.229200
2500,2.077400
3000,1.945800
3500,1.848700


TrainOutput(global_step=3971, training_loss=2.3614908341825895, metrics={'train_runtime': 10441.9563, 'train_samples_per_second': 97.363, 'train_steps_per_second': 0.38, 'total_flos': 1.3600631509622784e+17, 'train_loss': 2.3614908341825895, 'epoch': 1.0})

Now we save everything and upload it all

In [ ]:
# # safe: upload to transfer.sh
# !zip -r pretrain_data.zip pretrain
# !curl --upload-file ./pretrain_data.zip -o curl.out https://transfer.sh/pretrain_data.zip | cat curl.out
# !cat curl.out

In [ ]:
# better: upload to hub
model.push_to_hub(
    repo_id="jedwang/bert-base-split-chinese",
    revision="quart",
    commit_description="Completed pretraining!"
)

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jedwang/bert-base-split-chinese/commit/78ac6b1db75e3170dde831672ef3b8d20dd26b09', commit_message='Upload RobertaForMaskedLM', commit_description='Completed pretraining!', oid='78ac6b1db75e3170dde831672ef3b8d20dd26b09', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline

mask_filler = pipeline("fill-mask",
                       model="jedwang/bert-base-split-chinese",
                       revision="quart_x3",
                       tokenizer=tokenizer)
mask_filler("北京很[MASK][MASK][MASK]。", top_k=5)

config.json:   0%|          | 0.00/675 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

[[{'score': 0.032941777259111404,
   'token': 12438,
   'token_str': '学',
   'sequence': '[CLS]北京很学[MASK][MASK]。[SEP]'},
  {'score': 0.02287372574210167,
   'token': 1632,
   'token_str': '',
   'sequence': '[CLS]北京很[MASK][MASK]。[SEP]'},
  {'score': 0.012948013842105865,
   'token': 10959,
   'token_str': '团',
   'sequence': '[CLS]北京很团[MASK][MASK]。[SEP]'},
  {'score': 0.01227052416652441,
   'token': 1639,
   'token_str': '氵',
   'sequence': '[CLS]北京很氵[MASK][MASK]。[SEP]'},
  {'score': 0.011282827705144882,
   'token': 1636,
   'token_str': '木',
   'sequence': '[CLS]北京很木[MASK][MASK]。[SEP]'}],
 [{'score': 0.08869028091430664,
   'token': 15317,
   'token_str': '大学',
   'sequence': '[CLS]北京很[MASK]大学[MASK]。[SEP]'},
  {'score': 0.04041432961821556,
   'token': 6867,
   'token_str': '巷',
   'sequence': '[CLS]北京很[MASK]巷[MASK]。[SEP]'},
  {'score': 0.03923241049051285,
   'token': 10959,
   'token_str': '团',
   'sequence': '[CLS]北京很[MASK]团[MASK]。[SEP]'},
  {'score': 0.03766278922557831,
   'tok

I'll repeat the training process two more times

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    # eval_dataset=pretrain_dataset["eval"], # we're doing 0 eval
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Step,Training Loss
500,2.111500
1000,2.118000
1500,2.013800
2000,1.915700
2500,1.821500
3000,1.742300
3500,1.672400


TrainOutput(global_step=3971, training_loss=1.8776250650467845, metrics={'train_runtime': 10523.8656, 'train_samples_per_second': 96.606, 'train_steps_per_second': 0.377, 'total_flos': 1.34820145658069e+17, 'train_loss': 1.8776250650467845, 'epoch': 1.0})

In [ ]:
model.push_to_hub(
    repo_id="jedwang/bert-base-split-chinese",
    revision="quart_x2",
    commit_description="Completed pretraining x2!"
)

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jedwang/bert-base-split-chinese/commit/66e5b385b342005b096e485e2bd6c54b476968d8', commit_message='Upload RobertaForMaskedLM', commit_description='Completed pretraining x2!', oid='66e5b385b342005b096e485e2bd6c54b476968d8', pr_url=None, pr_revision=None, pr_num=None)

Once more

In [ ]:
from transformers import AutoModelForMaskedLM

model = model if 'model' in globals() else AutoModelForMaskedLM.from_pretrained(
    "jedwang/pretrain"
)

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    # eval_dataset=pretrain_dataset["eval"], # we're doing 0 eval
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.746600
1000,1.764300
1500,1.707400
2000,1.650500
2500,1.591900
3000,1.596200
3500,1.544500


TrainOutput(global_step=3971, training_loss=1.6387064272617098, metrics={'train_runtime': 10412.986, 'train_samples_per_second': 97.634, 'train_steps_per_second': 0.381, 'total_flos': 1.34820145658069e+17, 'train_loss': 1.6387064272617098, 'epoch': 1.0})

In [ ]:
model.push_to_hub(
    repo_id="jedwang/bert-base-split-chinese",
    revision="quart_x3",
    commit_description="Completed pretraining x3!"
)

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jedwang/bert-base-split-chinese/commit/27224ecf263fbaf281946678cfd8707df8b4ba98', commit_message='Upload RobertaForMaskedLM', commit_description='Completed pretraining x3!', oid='27224ecf263fbaf281946678cfd8707df8b4ba98', pr_url=None, pr_revision=None, pr_num=None)

~~I'll do once more since I can~~ I'll run out of compute hours